In [2]:
import sys
import os
import toml

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split

import pandas as pd
import numpy as np
import scipy.optimize as SciOpt
from tqdm.notebook import tqdm

sys.path.append(os.path.join(sys.path[0], '../..'))

from data.io import Reader
from data.kcost_dataset import KCostDataSetSplit, KCostDataSet
from model.kcost import KCostModel, KCostModelAlpha
import lsm.cost as CostFunc

In [3]:
cfg = toml.load('../../config/training.toml')
env = Reader.read_config('../../config/endure.toml')
cf = CostFunc.EndureKHybridCost(**env['system'])
cfg, env

({'model': 'KCostModel',
  'log': {'level': 'INFO', 'name': 'endure'},
  'io': {'data_dir': '/data',
   'train_dir': 'train_data',
   'test_dir': 'test_data',
   'model_dir': 'models'},
  'static_params': {'max_levels': 16,
   'max_size_ratio': 50,
   'mean_bias': [4.75, 0.5, 0.5, 0.5, 0.5],
   'std_bias': [2.74, 0.3, 0.3, 0.3, 0.3],
   'out_dims': 4},
  'hyper_params': {'num_cont_vars': 5,
   'num_cate_vars': 17,
   'hidden_layers': 2,
   'embedding_size': 17},
  'train': {'max_epochs': 128,
   'batch_size': 1024,
   'learning_rate': 0.001,
   'learning_rate_decay': 0.9,
   'early_stop_num': 2,
   'shuffle': True},
  'validate': {'batch_size': 16384, 'percent': 0.1, 'shuffle': False}},
 {'project': {'name': 'ENDURE',
   'log_level': 'INFO',
   'experiments': ['CostSurfaceEp']},
  'io': {'data_dir': '/data',
   'cold_data_dir': '/data',
   'train_dir_name': 'train_data',
   'test_dir_name': 'test_data'},
  'system': {'B': 4, 'phi': 1, 's': 0.0, 'E': 1024, 'H': 10, 'N': 100000000},
  'e

In [4]:
model = KCostModelAlpha(cfg, device='cpu')
model.state_dict()
model_data = torch.load('/data/models/kcost_min.model') 
model.eval()
model.load_state_dict(model_data)

<All keys matched successfully>

In [7]:
%%time
test_path = '/data/test_data'
test_files = [os.path.join(test_path, f'test_{idx:03}.feather') for idx in range(2)]
test_data = KCostDataSet(cfg, test_files)
test = DataLoader(test_data, batch_size=1)

CPU times: user 50.4 ms, sys: 31 ms, total: 81.4 ms
Wall time: 33.9 ms


In [49]:
z0, z1, q, w = (0.33, 0.33, 0.33, 0.01)
analytical_cf = lambda h, T, K: cf.calc_cost(h, T, K, z0, z1, q, w)
def cost_func_objective(args):
    h, T, K = args[0], args[1], args[2:-4]
    return analytical_cf(h, T, K)

def one_hot_encode(vec):
    categories = torch.flatten(nn.functional.one_hot(torch.tensor(vec).to(torch.int64), num_classes=cfg['static_params']['max_size_ratio']))
    return categories

def cost_func_learned(args):
    h = args[0]
    cate_inputs = one_hot_encode(args[1:])
    cont_inputs = torch.tensor([h, z0, z1, q, w]).float()
    inputs = torch.cat((cont_inputs, cate_inputs), dim=-1).reshape(1, -1)
    with torch.no_grad():
        pred = model(inputs)
        pred = pred.sum().item()
    
#     print(f'{args=}, {inputs.shape=}, {pred=}')
    print(f'{h=}')
    return pred

In [53]:
T_UPPER_LIM, T_LOWER_LIM = (50, 2)
one_mib_in_bits = 1024 * 1024 * 8
H_UPPER_LIM = 9.5

h_initial = 1.
T_initial = 2.
K_initial = [1.] * cfg['static_params']['max_levels']

bounds = [(0, H_UPPER_LIM), (T_LOWER_LIM, T_UPPER_LIM)] + [(T_LOWER_LIM - 1, T_UPPER_LIM - 1)] * cfg['static_params']['max_levels']
min_cost = np.inf
design = {}
minimizer_kwargs = {
    'method' : 'SLSQP',
    'bounds' : bounds,
    'options': {'ftol': 1e-2, 'eps': 1e-6, 'disp': True}}
#     'options': {'ftol': 1e-9, 'disp': True}}
x0 = np.array([h_initial, T_initial] + K_initial)

In [54]:
analytical_sol = SciOpt.minimize(
    fun=cost_func_objective,
    x0=x0,
    **minimizer_kwargs
)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.757976345103807
            Iterations: 5
            Function evaluations: 96
            Gradient evaluations: 5


In [55]:
learned_sol = SciOpt.minimize(
    fun=cost_func_learned,
    x0=x0,
    **minimizer_kwargs
)

h=1.0
h=1.000001
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=1.0
h=3.384185791211764
h=3.3841867912117642
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
h=3.384185791211764
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.228340148925781
            Iterations: 2
            Function evaluations: 38
            Gradient evaluations: 2


In [56]:
learned_sol

     fun: 4.228340148925781
     jac: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
 message: 'Optimization terminated successfully'
    nfev: 38
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([3.38418579, 2.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        ])

In [57]:
analytical_sol

     fun: 1.757976345103807
     jac: array([-0.01987533,  0.00736854,  0.31456106,  0.31560044,  0.32283628,
        0.35992855,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])
 message: 'Optimization terminated successfully'
    nfev: 96
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([7.6019589 , 9.42905144, 1.        , 1.        , 1.        ,
       1.        , 1.13609266, 1.5157039 , 2.40732031, 2.54918536,
       2.99272069, 3.53244076, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        ])